In [1]:
import sys
print("Python executable:", sys.executable)


Python executable: c:\Users\maria\OneDrive\Documents\BigData\.venv\Scripts\python.exe


In [ ]:
import sys
!{sys.executable} -m pip install pandas chromadb sentence-transformers tqdm

In [3]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

print("pandas:", pd.__version__)
print("chromadb:", chromadb.__version__)

c:\Users\maria\OneDrive\Documents\BigData\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pandas: 2.3.0
chromadb: 1.0.12


In [4]:
import pandas as pd

# 2.1) Ajuste o caminho se necessário
csv_path = "dataset.csv"

# 2.2) Leitura pulando linhas mal-formatadas
df = pd.read_csv(
    csv_path,
    sep=";",
    encoding="utf-8",
    on_bad_lines="skip",
    parse_dates=["DataCriacao", "DataResposta"],
    dayfirst=True
)

print(f"📊 Linhas carregadas: {len(df)}")
df.head(3)


📊 Linhas carregadas: 355436


,Tipo,Orgao,DataCriacao,Assunto,Subdivisao,Situacao,Logradouro,Bairro,Regional,DataResposta,Origem,Unnamed: 11
0,Solicitação,Fundação de Ação Social,2025-06-07,Abordagem social de rua - Adulto e Idoso,Adulto desaparecido,Respondido,ESTRADA GUILHERME WEIGERT,SANTA CANDIDA,Unidade Regional Boa Vista,2025-06-07,Telefone,NaN
1,Solicitação,Fundação de Ação Social,2025-06-07,Abordagem social de rua - Adulto e Idoso,Pessoas/famílias em desabrigo na rua,Respondido,TERMINAL BOQUEIRÃO,BOQUEIRAO,Unidade Regional Boqueirao,2025-06-07,Telefone,NaN
2,Solicitação,Secretaria Municipal de Trânsito,2025-06-07,Trânsito,Fiscalização de bloqueio de pista,Respondido,Rua Frederico Cantarelli,BIGORRILHO,Unidade Regional Matriz,2025-06-07,Mobile,NaN


In [5]:
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

print("✅ chromadb version:", chromadb.__version__)


✅ chromadb version: 1.0.12


In [6]:
# 3.2) Converte datas para string
df["DataCriacao_str"] = df["DataCriacao"].dt.strftime("%Y-%m-%d").fillna("").astype(str)
df["DataResposta_str"] = df["DataResposta"].dt.strftime("%Y-%m-%d").fillna("").astype(str)

# 3.3) Monta o 'text' agregando todos os campos
df["text"] = df.apply(lambda r: (
    f"Tipo: {r['Tipo']}; Órgão: {r['Orgao']}; Assunto: {r['Assunto']}; "
    f"Subdivisão: {r['Subdivisao']}; Situação: {r['Situacao']}; "
    f"Logradouro: {r['Logradouro']}; Bairro: {r['Bairro']}; "
    f"Regional: {r['Regional']}; Origem: {r['Origem']}; "
    f"DataCriação: {r['DataCriacao_str']}; DataResposta: {r['DataResposta_str']}"
), axis=1)

# 3.4) Monta um 'context' mais rico, incluindo tema, bairro e status
df["context"] = df.apply(lambda r: (
    f"{r['Assunto']} – Subdivisão: {r['Subdivisao']}; Bairro: {r['Bairro']}; Situação: {r['Situacao']}"
), axis=1)

# 3.5) Gere embeddings (o resto segue igual)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeds = model.encode(df["text"].tolist(), show_progress_bar=True).tolist()


Batches: 100%|██████████| 11108/11108 [1:08:21<00:00,  2.71it/s]


In [ ]:
# 3.5) Inicializa cliente persistente e recria coleção
db_path = "./ouvidoria156_bd"
client = chromadb.PersistentClient(path=db_path)

col_name = "collection_v2"  # ajuste para o nome correto da sua coleção

# Remove coleção antiga, se existir
if col_name in [c.name for c in client.list_collections()]:
    client.delete_collection(col_name)

# Cria (ou obtém) a coleção nova
col = client.create_collection(name=col_name, get_or_create=True)

# 3.6) Inserção em batches, já com metadados em string
docs        = df["text"].tolist()
embeds_list = embeds     
ids         = [str(i) for i in df.index]
batch_size  = 5000

for i in tqdm(range(0, len(docs), batch_size), desc=f"Inserindo na {col_name}"):
    chunk = df.iloc[i : i + batch_size]
    metas = [
        {
          "Tipo":         str(r.Tipo),
          "Órgão":        str(r.Orgao),
          "Assunto":      str(r.Assunto),
          "Subdivisão":   str(r.Subdivisao),
          "Situação":     str(r.Situacao),
          "Logradouro":   str(r.Logradouro),
          "Bairro":       str(r.Bairro),
          "Regional":     str(r.Regional),
          "Origem":       str(r.Origem),
          "DataCriacao":  r.DataCriacao_str,
          "DataResposta": r.DataResposta_str,
          "context":      r.context
        }
        for r in chunk.itertuples()
    ]
    col.add(
        documents  = docs[i : i + batch_size],
        embeddings = embeds_list[i : i + batch_size],
        metadatas  = metas,
        ids        = ids[i : i + batch_size]
    )

print(f"✅ Indexação concluída em: {db_path}, coleção: {col_name}")

Inserindo na collection_v2: 100%|██████████| 72/72 [05:16<00:00,  4.39s/it]

✅ Indexação concluída em: ./ouvidoria156_bd, coleção: collection_v2


In [44]:
from sentence_transformers import SentenceTransformer
import time
# 1. Query vetorial
query = "pedido árvore obstruindo via"
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
q_emb = model.encode([query]).tolist()
# 2. Início da contagem de tempo
start_time = time.time()
# 3. Consulta com filtro
results = collection.query(
    query_embeddings=q_emb,
    # where={"Bairro": {"$eq": "SANTA CANDIDA"}},  
    n_results=5,
    include=["documents", "metadatas"]
)
# 4. Fim da contagem de tempo
end_time = time.time()
elapsed = end_time - start_time
# 5. Resultado
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print("📝", doc)
    print("🔎", meta)
    print()

print(f"⏱️ Tempo total de execução: {elapsed:.3f} segundos")

📝 Tipo: Solicitação; Órgão: Secretaria Municipal do Meio Ambiente; Assunto: Árvore; Subdivisão: Árvore ou galho caído – sem bloqueio de passagem; Situação: Concluído; Logradouro: AVENIDA JORNALISTA ADERBAL GAERTNER STRESSER; Bairro: CAJURU; Regional: Unidade Regional Cajuru; Origem: Telefone; DataCriação: 2025-02-06; DataResposta: 2025-02-07
🔎 {'Órgão': 'Secretaria Municipal do Meio Ambiente', 'Tipo': 'Solicitação', 'Regional': 'Unidade Regional Cajuru', 'Situação': 'Concluído', 'Subdivisão': 'Árvore ou galho caído – sem bloqueio de passagem', 'Bairro': 'CAJURU', 'context': 'Árvore – Subdivisão: Árvore ou galho caído – sem bloqueio de passagem; Bairro: CAJURU; Situação: Concluído', 'DataResposta': '2025-02-07', 'Origem': 'Telefone', 'Logradouro': 'AVENIDA JORNALISTA ADERBAL GAERTNER STRESSER', 'Assunto': 'Árvore', 'DataCriacao': '2025-02-06'}

📝 Tipo: Solicitação; Órgão: Secretaria Municipal do Meio Ambiente; Assunto: Árvore; Subdivisão: Árvore ou galho caído – sem bloqueio de passagem